In [1]:
from polish_parser import Parser

my_parser = Parser()

In [2]:
my_parser.parse("p")

Position: 0
Length: 1
Expecting: [pies, pan, pilot, ptak, policjant, pisarz, pilot, piosenka, psy, panowie, piloci, ptaki, policjanci, pisarze, piloci, piosenki]
Reason: Unrecognized word